## Pypspark with financial log  dataset 
Dataset link: https://www.kaggle.com/datasets/ealaxi/paysim1/

In [1]:
# %load_ext nb_black
# !pip install pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
spark = SparkSession.builder.getOrCreate()

23/12/15 06:31:29 WARN Utils: Your hostname, kudsit-HP-ProOne-600-G6-22-All-in-One-PC resolves to a loopback address: 127.0.1.1; using 10.10.28.180 instead (on interface eno1)
23/12/15 06:31:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/12/15 06:31:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [41]:
# 1. Import required depencies n packages
# 2. Load the csv file
# 3. Spilt the dataset
# 4. Check the datatypes of the attributes
# 5. String Indexer
# 6. One Hot encoding 
# 7. vector Assembler 
# 8. Fitting the model 
# 9. Get the metrics


In [5]:
import os
from os.path import isfile, join 

# loc = os.path.abspath("")
# data_loc = f"{loc}/data/"
# # spark = SparkSession.builder.appName("Loading CSV").getOrCreate()

In [9]:
#replace file_path with the file path and file name with the actual file name of the dataset 
df= spark.read.csv("file_path/file_name.csv", inferSchema= True, header = True)
# df = spark.read.csv(
#     f"{data_loc}financial_dataset.csv", inferSchema = True, header = True)

In [10]:

df.printSchema()

root
 |-- step: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- nameOrig: string (nullable = true)
 |-- oldbalanceOrg: double (nullable = true)
 |-- newbalanceOrig: double (nullable = true)
 |-- nameDest: string (nullable = true)
 |-- oldbalanceDest: double (nullable = true)
 |-- newbalanceDest: double (nullable = true)
 |-- isFraud: integer (nullable = true)
 |-- isFlaggedFraud: integer (nullable = true)



In [12]:
df.show(5)

+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|    type|  amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|   1| PAYMENT| 9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|             0|
|   1| PAYMENT| 1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|             0|
|   1|TRANSFER|   181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|      1|             0|
|   1|CASH_OUT|   181.0| C840083671|        181.0|           0.0|  C38997010|       21182.0|           0.0|      1|             0|
|   1| PAYMENT|11668.14|C2048537720|      41554.0|      29885.86|M1230701703|      

In [13]:
df = df.select("type", "amount", "oldbalanceOrg", "newbalanceOrig", "isFraud")

In [14]:
"""Train Test Split"""
train, test = df.randomSplit((0.7,  0.3), seed =7)

In [15]:
print(f"Train set length: {train.count()} records")
print(f"Test set length: {test.count()} records")

Train set length: 4452656 records


Test set length: 1909964 records


In [16]:
train.show(2)

+-------+------+-------------+--------------+-------+
|   type|amount|oldbalanceOrg|newbalanceOrig|isFraud|
+-------+------+-------------+--------------+-------+
|CASH_IN|  5.66|   5061561.06|    5061566.72|      0|
|CASH_IN| 13.86|   6868100.18|    6868114.04|      0|
+-------+------+-------------+--------------+-------+
only showing top 2 rows



In [17]:
"""Dtypes"""
"""In this dataset, any column of type string is treated as categorical feature, but sometimes we have 
numerical feature we want to treat as categorical or vice versa. Thus we need to identitfy which columns are
categorical and which columns are numerical."""
train.dtypes

[('type', 'string'),
 ('amount', 'double'),
 ('oldbalanceOrg', 'double'),
 ('newbalanceOrig', 'double'),
 ('isFraud', 'int')]

In [18]:
catCols = [x for (x, dataType) in train.dtypes if dataType == "string"]
numCols = [x for (x, dataType) in train.dtypes if ((dataType == "double") & (x!= "isFraud")) ]

In [19]:
print(catCols)
print(numCols)

['type']
['amount', 'oldbalanceOrg', 'newbalanceOrig']


In [20]:
train.agg(F.countDistinct("type")).show() #no of categories 

+-----------+
|count(type)|
+-----------+
|          5|
+-----------+



In [21]:
train.groupBy("type").count().show()

23/12/15 06:42:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/15 06:42:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/15 06:42:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/15 06:42:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/15 06:42:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/15 06:42:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/15 06:42:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/15 06:42:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/15 06:42:46 WARN RowBasedKeyValueBatch: Calling spill() on

In [22]:
from pyspark.ml.feature import (
OneHotEncoder,
StringIndexer)


In [23]:
string_indexer  = [StringIndexer(inputCol=x, outputCol = x+ "_StringIndexer", handleInvalid= "skip")
                  for x in catCols]


In [24]:
string_indexer

[StringIndexer_a39e6c6da99c]

In [25]:
"""One hot encoding """
one_hot_encoder  =[
    OneHotEncoder(
        inputCols= [f"{x}_StringIndexer" for x in catCols],
        outputCols= [f"{x}_OneHotEncoder" for x in catCols]          
                 )]

In [26]:
one_hot_encoder

[OneHotEncoder_41334fae80ff]

In [27]:
#in spark we have to send all of the feature as a Vector  Assembler does this 
from pyspark.ml.feature import VectorAssembler

In [28]:
assemblerInput = [x for x in numCols]
assemblerInput += [f"{x}_OneHotEncoder" for x in catCols]

In [29]:
assemblerInput

['amount', 'oldbalanceOrg', 'newbalanceOrig', 'type_OneHotEncoder']

In [30]:
vector_assembler = VectorAssembler(
    inputCols = assemblerInput, outputCol = "VectorAssembler_features")

In [31]:
#Pyspark Pipeline
stages = []
stages += string_indexer
stages += one_hot_encoder
stages += [vector_assembler]

In [32]:
stages #similar to pipeline in scikit learn

[StringIndexer_a39e6c6da99c,
 OneHotEncoder_41334fae80ff,
 VectorAssembler_661f863cd3a6]

In [33]:
from pyspark.ml import Pipeline
pipeline = Pipeline().setStages(stages)
model = pipeline.fit(train)

pp_df = model.transform(test)

In [34]:
pp_df.select(
    "type", "amount", "oldbalanceOrg", "newbalanceOrig","VectorAssembler_features" ).show(truncate = False)

+-------+------+-------------+--------------+---------------------------------------------------+
|type   |amount|oldbalanceOrg|newbalanceOrig|VectorAssembler_features                           |
+-------+------+-------------+--------------+---------------------------------------------------+
|CASH_IN|14.4  |1.143460813E7|1.143462253E7 |[14.4,1.143460813E7,1.143462253E7,0.0,0.0,1.0,0.0] |
|CASH_IN|15.59 |1.64294897E7 |1.642950528E7 |[15.59,1.64294897E7,1.642950528E7,0.0,0.0,1.0,0.0] |
|CASH_IN|21.15 |2729078.29   |2729099.44    |[21.15,2729078.29,2729099.44,0.0,0.0,1.0,0.0]      |
|CASH_IN|22.67 |405940.0     |405962.67     |[22.67,405940.0,405962.67,0.0,0.0,1.0,0.0]         |
|CASH_IN|24.2  |1.803342949E7|1.803345368E7 |[24.2,1.803342949E7,1.803345368E7,0.0,0.0,1.0,0.0] |
|CASH_IN|35.47 |3796691.21   |3796726.68    |[35.47,3796691.21,3796726.68,0.0,0.0,1.0,0.0]      |
|CASH_IN|38.77 |6210013.72   |6210052.49    |[38.77,6210013.72,6210052.49,0.0,0.0,1.0,0.0]      |
|CASH_IN|45.02 |2745

In [35]:
"""Logistic Regression """
from pyspark.ml.classification import LogisticRegression

In [36]:
data = pp_df.select(
    F.col("VectorAssembler_features").alias("features"),
    F.col("isFraud").alias("label"))

In [37]:
data.show(5, truncate = False)

+--------------------------------------------------+-----+
|features                                          |label|
+--------------------------------------------------+-----+
|[14.4,1.143460813E7,1.143462253E7,0.0,0.0,1.0,0.0]|0    |
|[15.59,1.64294897E7,1.642950528E7,0.0,0.0,1.0,0.0]|0    |
|[21.15,2729078.29,2729099.44,0.0,0.0,1.0,0.0]     |0    |
|[22.67,405940.0,405962.67,0.0,0.0,1.0,0.0]        |0    |
|[24.2,1.803342949E7,1.803345368E7,0.0,0.0,1.0,0.0]|0    |
+--------------------------------------------------+-----+
only showing top 5 rows



In [38]:
%%time
model = LogisticRegression().fit(data)

CPU times: user 11 ms, sys: 5.94 ms, total: 17 ms
Wall time: 12.4 s


In [39]:
model.summary.areaUnderROC

0.9930925361016105

In [40]:
model.summary.pr.show()

+-------------------+-------------------+
|             recall|          precision|
+-------------------+-------------------+
|                0.0| 0.8992654774396642|
|0.35887772194304857| 0.8992654774396642|
|0.47571189279731996| 0.6558891454965358|
| 0.5510887772194305| 0.5240939864595778|
| 0.5954773869346733|0.43221884498480245|
| 0.6256281407035176|0.36716637994593265|
| 0.6612227805695142| 0.3257013201320132|
| 0.6888609715242882| 0.2923405011551448|
|  0.714824120603015|0.26646893537308775|
| 0.7349246231155779|0.24425887265135698|
| 0.7546063651591289|0.22626820693119035|
| 0.7713567839195979|0.21068283197986962|
| 0.7876884422110553|0.19754253308128544|
| 0.7981574539363484|0.18503057955538296|
|  0.804857621440536|0.17346570397111913|
| 0.8069514237855946|0.16249262163757483|
| 0.8111390284757118|0.15325579555344568|
| 0.8136515912897823|0.14480548516917574|
| 0.8165829145728644|0.13735296189335774|
| 0.8207705192629816|0.13087606837606838|
+-------------------+-------------